In [63]:
from google.colab import files
uploaded = files.upload()


Saving Transformer_model.py to Transformer_model (2).py


In [64]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [66]:
import numpy as np
import os
import torch
from torch import nn
#from transformer import Transformer
from Transformer_model import Transformer
from torch.utils.data import Dataset, DataLoader

In [67]:
START_TOKEN = '<START>'
PADDING_TOKEN = '<PADDING>'
END_TOKEN = '<END>'

# Vocabularies
hindi_vocab = [
    START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '<', '=', '>', '?', '।', '॥',
    'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ए', 'ऐ', 'ओ', 'औ',
    'क', 'ख', 'ग', 'घ', 'ङ',
    'च', 'छ', 'ज', 'झ', 'ञ',
    'ट', 'ठ', 'ड', 'ढ', 'ण',
    'त', 'थ', 'द', 'ध', 'न',
    'प', 'फ', 'ब', 'भ', 'म',
    'य', 'र', 'ल', 'व', 'श', 'ष', 'स', 'ह',
    'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'े', 'ै', 'ो', 'ौ', '्', 'ॐ',
    '०', '१', '२', '३', '४', '५', '६', '७', '८', '९', PADDING_TOKEN, END_TOKEN]

english_vocab = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
                      '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                      ':', '<', '=', '>', '?', '@',
                      '[', '\\', ']', '^', '_', '`',
                      'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                      'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
                      'y', 'z',
                      '{', '|', '}', '~', PADDING_TOKEN, END_TOKEN]



In [68]:
index_eng = { i:c for i,c in enumerate(english_vocab)}
eng_index = { c:i for i,c in enumerate(english_vocab)}
index_hin = { i:c for i,c in enumerate(hindi_vocab)}
hin_index = { c:i for i,c in enumerate(hindi_vocab)}
print(len(hin_index))
print(len(eng_index))

102
71


In [69]:
eng_file = '/content/drive/MyDrive/train.en'
hin_file = '/content/drive/MyDrive/train.hi'

def read_lines(file_name):
    with open(file_name, 'r') as file:
        sentence = file.readlines()
        return sentence

eng_sentence = read_lines(eng_file)
hin_sentence = read_lines(hin_file)
print(len(eng_sentence))
# storing sentences to a list of max length
max_len = 1800000
eng_sentence = eng_sentence[:max_len]
hin_sentence = hin_sentence[:max_len]


10125706


In [70]:
hin_sentence[-5:]

['गर्मी / फ्रांस में पहली बार पारा 45 डिग्री पार, सड़कों पर फव्वारे लगाए गए. पूरे यूरोप में गर्मी के लिए अलर्ट\n',
 'डाक्टर व्दारा, वेल वुमन क्लिनिक या परिवार नियोजन क्लिनिक में छेस्छ किया जा सकता है।\n',
 'सारे आसमान और ज़मीन का मवविद (बनाने वाला) है उसके कोई लड़का क्योंकर हो सकता है जब उसकी कोई बीबी ही नहीं है और उसी ने हर चीज़ को पैदा किया और वही हर चीज़ से खूब वाक़िफ है\n',
 'क्कालांतर में कोच्चि राज्य के रजवाड़े में परिवर्तित होने के क्साथ ही ये डच एवं ब्रिटिश के नियन्त्रण में आ गया।\n',
 'गुजरात के गिर वन में, उनके भोजन का बड़ा प्रतिशत पेड़ों पर से गिरा हुआ फल ज़िज़िफस होता है।\n']

In [71]:
eng_sentence = [sentence.rstrip('\n').lower() for sentence in eng_sentence]
hin_sentence = [sentence.rstrip('\n').lower() for sentence in hin_sentence]
print(len(hin_sentence))

1800000


In [72]:
eng_sentence[-5:]

['home / world news / heat tops 45 degrees in france for first time as deadly heatwave roasts europe',
 'a doctor will examine you in a well women clinic or family planning clinic.',
 'the creator of the heavens and the earth -- how should he have a son, seeing that he has no consort, and he created all things, and he has knowledge of everything?',
 'the city was later occupied by the dutch and the british, with the kingdom of cochin becoming a princely state.',
 'in the gir forest of gujarat, a large percentage of their food is made up of the fallen berries of zizyphus.']

In [73]:
# max sequence lenght in a sentence :(T)
# Computing 96th percentile to hinow the max sequence length as we will be uisng 20000 sentenses for training the model.
# sequence length can be tuned (T)
# input sequnece can be tuned

def max_len(language):
    P = 99
    length = [len(i) for i in language]
    maximum_len = max(length)
    percentile =  np.percentile(length, P)
    return maximum_len , percentile

print('max lenght & Percentile of English Sentences:', max_len(eng_sentence))
print('max lenght & Percentile of Hindi Sentences:', max_len(hin_sentence))


max lenght & Percentile of English Sentences: (14820, 352.0)
max lenght & Percentile of Hindi Sentences: (7216, 353.0)


In [74]:
max_sequence_length = 360

def is_valid_tokens(sentence, vocab):
    for token in list(set(sentence)):
        if token not in vocab:
            return False
    return True

def is_valid_length(sentence, max_sequence_length):
    return len(list(sentence)) < (max_sequence_length - 1) # to re-add the end token we leave 1 space

valid_sentence_len = []
for index in range(len(hin_sentence)):
    hindi_sentence, english_sentence = hin_sentence[index], eng_sentence[index]
    if is_valid_length(hindi_sentence, max_sequence_length) \
      and is_valid_length(english_sentence, max_sequence_length) \
      and is_valid_tokens(hindi_sentence, hindi_vocab):
        valid_sentence_len.append(index)

print(f"Number of sentences: {len(hin_sentence)}")
print(f"Number of valid sentences: {len(valid_sentence_len)}")

Number of sentences: 1800000
Number of valid sentences: 155818


In [75]:
valid_sentence_len

[3,
 13,
 18,
 21,
 38,
 55,
 59,
 62,
 77,
 83,
 95,
 112,
 122,
 123,
 137,
 153,
 155,
 174,
 177,
 188,
 201,
 237,
 244,
 274,
 276,
 278,
 297,
 300,
 316,
 321,
 325,
 334,
 336,
 339,
 342,
 345,
 380,
 386,
 393,
 402,
 440,
 448,
 455,
 460,
 461,
 465,
 483,
 516,
 521,
 542,
 546,
 548,
 559,
 569,
 571,
 572,
 583,
 585,
 591,
 597,
 598,
 601,
 640,
 644,
 655,
 665,
 680,
 683,
 703,
 707,
 708,
 710,
 734,
 737,
 746,
 775,
 777,
 788,
 799,
 822,
 829,
 831,
 832,
 842,
 847,
 849,
 855,
 865,
 869,
 885,
 913,
 936,
 941,
 948,
 954,
 963,
 980,
 985,
 988,
 997,
 1004,
 1006,
 1027,
 1100,
 1102,
 1103,
 1105,
 1108,
 1109,
 1115,
 1123,
 1126,
 1128,
 1143,
 1149,
 1186,
 1187,
 1212,
 1226,
 1230,
 1255,
 1267,
 1268,
 1276,
 1298,
 1313,
 1314,
 1323,
 1335,
 1337,
 1339,
 1354,
 1355,
 1363,
 1375,
 1378,
 1379,
 1425,
 1435,
 1443,
 1451,
 1455,
 1456,
 1458,
 1470,
 1490,
 1518,
 1528,
 1537,
 1558,
 1579,
 1580,
 1583,
 1585,
 1590,
 1607,
 1670,
 1676,
 1689,

In [76]:
hindi_sentences = [hin_sentence[i] for i in valid_sentence_len]
english_sentences = [eng_sentence[i] for i in valid_sentence_len]
print(hindi_sentences[0])
print(english_sentences[0])

आस्ट्रेलिया के खिलाफ वनडे टीम की कमान मिताली को
mithali to anchor indian team against australia in odis


In [77]:
d_model = 512
batch_size = 30
#ffn_hidden is the number of  neurons in the hidden layer of Feed Forward Neural Network (Fully Connected)
ffn_hidden = 2048
num_head = 8
drop_prob = 0.1
num_layers = 1
max_sequence_length = 360
d_output = len(hindi_vocab)


transformer = Transformer(d_model,
                        ffn_hidden,
                        num_head,
                         drop_prob,
                         num_layers,
                         max_sequence_length,
                         d_output,
                          eng_index,
                          hin_index,
                          START_TOKEN,
                          END_TOKEN,
                          PADDING_TOKEN)

In [78]:
transformer

Transformer(
  (encoder): Encoder(
    (sentence_embedding): SentenceEmbedding(
      (embedding): Embedding(71, 512)
      (positional_encoder): PositionalEncoding()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): SequentialEncoder(
      (0): EncoderLayer(
        (attention): MultiHeadAttention(
          (qkv_layer): Linear(in_features=512, out_features=1536, bias=True)
          (linear_layer): Linear(in_features=512, out_features=512, bias=True)
        )
        (norm1): LayerNormalization()
        (dropout1): Dropout(p=0.1, inplace=False)
        (ffn): PositionWiseFeedForward(
          (linear1): Linear(in_features=512, out_features=2048, bias=True)
          (linear2): Linear(in_features=2048, out_features=512, bias=True)
          (relu): ReLU()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (norm2): LayerNormalization()
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (decoder): Decoder(
    (sentence_embedding

In [79]:
class TextDataset(Dataset):
    def __init__(self, english_sentences, hindi_sentences):
        self.english_sentences = english_sentences
        self.hindi_sentences = hindi_sentences

    def __len__(self):
        return len(self.english_sentences)

    def __getitem__(self, index):
        return self.english_sentences[index], self.hindi_sentences[index]

In [80]:
dataset = TextDataset(english_sentences, hindi_sentences)


In [81]:
len(dataset)

155818

In [82]:
dataset[-5]

('young india should come forward to change the system.',
 'इस व्यवस्था को बदलने के लिए युवा भारत को आगे आना चाहिए.')

In [83]:
batch_size = 30
train_loader = DataLoader(dataset, batch_size)
iterator = iter(train_loader)

In [84]:
len(iterator)

5194

In [85]:
for batch_num, batch in enumerate(iterator):
    print(batch)
    if batch_num>3:
        break


[('mithali to anchor indian team against australia in odis', 'meanwhile, three people came there on a bike.', 'share videos', 'hence, we must make sure we ourselves dont become careless, not allow anyone else do so', 'the fir exhibit ka-19', 'the bench rejected the petition.', 'i must go.', 'people are sad and angry.', 'everything for germany.', 'three persons were arrested by the police.', "they've stopped.", 'all of them were from uttar pradesh.', 'circumcise therefore the foreskin of your heart, and be no more stiff-necked.', 'about 50 people were injured.', 'one had moved to the united states.', 'govt repeals articles 370. rajya sabha passes jammu and kashmir reorganisation bill', 'this will change the scenario.', 'this will ensure better returns for the farmer as well as industry.', 'asked what is that mindset, the ndb head said, the borrowers mindset is please understand our challenges and lend accordingly.', 'the accused was brought here by the punjab police from nabha jail.', '

In [86]:
# to prevent the padding tokens from contributing to the loss.
criterian = nn.CrossEntropyLoss( ignore_index = hin_index[PADDING_TOKEN]) #so that the loss calculation ignores cases where the target label is the padding token.

for params in transformer.parameters():
    if params.dim() > 1 : # initializing weight parameters for 2d matrix or more
        nn.init.xavier_uniform(params)
        # nn.init.xavier_uniform --> initializes the weights of the layer with values drawn from a uniform distribution within a specified range.
        #  It helps in better weight initialization, which can lead to more stable training and better convergence of neural network models.

optim = torch.optim.Adam(transformer.parameters(), lr = 1e-4) # Adam Optimizer


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)


cuda


<ipython-input-86-37c934e5bce7>:6: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(params)


In [88]:
NEG_INFTY = -1e9

def create_mask(eng_batch, hin_batch):
    num_sentences = len(eng_batch)
    look_ahead_mask = torch.full([max_sequence_length, max_sequence_length], True)
    look_ahead_mask = torch.triu(look_ahead_mask, diagonal = 1)
    encoder_padding_mask = torch.full([num_sentences,max_sequence_length, max_sequence_length], False)
    decoder_padding_mask_self_attention = torch.full([num_sentences,max_sequence_length, max_sequence_length], False)
    decoder_padding_mask_cross_attention = torch.full([num_sentences,max_sequence_length, max_sequence_length], False)

    for idx in range(num_sentences):
        eng_sentence_length, hin_sentence_length = len(eng_batch[idx]), len(hin_batch[idx]) # length of each sentence in a batch character wise(letters)
        eng_chars_to_padding_mask = np.arange(eng_sentence_length + 1, max_sequence_length) # indices needed to be padded for english sentences
        hin_chars_to_padding_mask = np.arange(hin_sentence_length + 1, max_sequence_length)# indices needed to be padded for hindi sentences
        # The padding mask is used to mask out certain positions in the input sequence where padding tokens are present
        encoder_padding_mask[idx, :, eng_chars_to_padding_mask] = True
        encoder_padding_mask[idx, eng_chars_to_padding_mask, :] = True
        decoder_padding_mask_self_attention[idx, : ,hin_chars_to_padding_mask] = True
        decoder_padding_mask_self_attention[idx, hin_chars_to_padding_mask, :] = True
        decoder_padding_mask_cross_attention[idx, : ,eng_chars_to_padding_mask] = True
        decoder_padding_mask_cross_attention[idx, hin_chars_to_padding_mask, :] = True
    encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY, 0)
    decoder_self_attention_mask = torch.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0)
    decoder_cross_attention_mask = torch.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0)
    return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask


In [91]:
from tqdm import tqdm
transformer.train()
#transformer.to(device)

total_loss = 0
num_epochs = 1

for epoch in range(num_epochs):
    print('Epoch', epoch)
    iterator = iter(train_loader)
    for batch_num, batch in tqdm(enumerate(iterator), total=len(iterator)):
        transformer.train()
        eng_batch, hin_batch = batch
        encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_mask(eng_batch, hin_batch)
        optim.zero_grad() # --> optim.zero_grad() is a method provided by PyTorch's optimizer classes (Adam used here)
        # that sets the gradients of all the model's parameters to zero.
        # It's an essential step to perform before computing gradients during backpropagation.
        hin_predictions = transformer(eng_batch,
                                      hin_batch,
                                      encoder_self_attention_mask,
                                      decoder_self_attention_mask,
                                      decoder_cross_attention_mask,
                                      encoder_start_token = False,
                                      encoder_end_token = False,
                                      decoder_start_token = True,
                                      decoder_end_token = True)
        labels = transformer.decoder.sentence_embedding.batch_tokenize(hin_batch, start_token = False, end_token = True)
        # criterian = nn.CrossEntropyLoss( ignore_index = hin_index[PADDING_TOKEN])
        loss = criterian(hin_predictions.view(-1, d_output),labels.view(-1)) # -1 is a place holder
        valid_indices = torch.where(labels.view(-1) == hin_index[PADDING_TOKEN], False, True) # <-- Binary Tensor
        loss = loss.sum()/valid_indices.sum() # <-- calculating the mean loss over the valid examples.
        loss.backward()
        # This line performs backpropagation to compute the gradients of the loss with respect to the model's parameters.
        #These gradients are essential for updating the model's parameters during the optimization step.
        optim.step() #<-- updates the model's parameters using an optimizer.
        # The optimizer uses the computed gradients to adjust the model's parameters in a way that minimizes the loss function
        if batch_num % 100 == 0:
            print(f'iteration {batch_num} : {loss.item()}')
            print(f"English: {eng_batch[0]}")
            print(f"Hindi Translation: {hin_batch[0]}")
            hin_sentence_predicted = torch.argmax(hin_predictions[0], axis= 1)
            predicted_sentence = ""
            for idx in hin_sentence_predicted:
                if idx == hin_index[END_TOKEN]:
                    break
                predicted_sentence += index_hin[idx.item()]
            print(f"Hindi Prediction: {predicted_sentence}")

save_path = '/content/drive/MyDrive/transformer_weights.pth'
torch.save({
    'model_state_dict': transformer.state_dict(),
    'optimizer_state_dict': optim.state_dict()
}, save_path)
print('weights saved in drive')

Epoch 0


  0%|          | 1/5194 [00:01<2:25:32,  1.68s/it]

iteration 0 : 0.0027983884792774916
English: mithali to anchor indian team against australia in odis
Hindi Translation: आस्ट्रेलिया के खिलाफ वनडे टीम की कमान मिताली को
Hindi Prediction: ससे                                                               क            ि ्   क ाा ्        ा                        ि ि       ि     र                        ्ा      ाय  ा      ि्  स    ।       स    ा  ाक   ा स  े    े       क       ्        े   ि                                                   ा                   ा्                                 


  2%|▏         | 101/5194 [02:46<2:25:47,  1.72s/it]

iteration 100 : 0.002830502111464739
English: what does the modi government have?
Hindi Translation: मोदी सरकार के पास क्या है विकल्प
Hindi Prediction: इस   के    का कार क सा क  ह क   ा     क  लल ल रसक ल         र   र  कनम   ल    ललत न  व  क त   क कररसपर ा           र    रककर क  कक   न फपर क  सनापसप सककम  ककअ क प   कप  ा   यन नमन ोन  वसवससस    कनक ल सकफअकवककससककक  नवयमक स स ससस    तस           क  कककस                 ल             ल             म      कक ए            ववतन                       ल स म लल स  स


...

iteration 5000 : 0.0017348754918202758
English: new york, october 11
Hindi Translation: कालका, 11 अक्तूबर (निस)
Hindi Prediction: स्र ्  क  कन्रा   (किस्


 98%|█████████▊| 5101/5194 [2:22:43<02:32,  1.64s/it]

iteration 5100 : 0.0016987138660624623
English: the movie is directed by ss rajamouli.
Hindi Translation: यह फिल्म एसएस राजामौली द्वारा निर्देशित है।
Hindi Prediction: इह सिल्म कके एकाजन  र  हियार  हेरफट श त है।


100%|██████████| 5194/5194 [2:25:21<00:00,  1.68s/it]

weights saved in drive


In [94]:
checkpoint = torch.load('/content/drive/MyDrive/transformer_weights.pth')
transformer.load_state_dict(checkpoint['model_state_dict'])
optim.load_state_dict(checkpoint['optimizer_state_dict'])
print("Model weights loaded.")


transformer.eval()
def translate(eng_sentence):
  eng_sentence = (eng_sentence,)
  hin_sentence = ("",)
  for word_counter in range(max_sequence_length):
    encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_mask(eng_sentence, hin_sentence)
    predictions = transformer(eng_sentence,
                              hin_sentence,
                              encoder_self_attention_mask,
                              decoder_self_attention_mask,
                              decoder_cross_attention_mask,
                              encoder_start_token=False,
                              encoder_end_token=False,
                              decoder_start_token=True,
                              decoder_end_token=False)
    next_token_prob_distribution = predictions[0][word_counter]
    next_token_index = torch.argmax(next_token_prob_distribution).item()
    next_token = index_hin[next_token_index]
    hin_sentence = (hin_sentence[0] + next_token, )
    if next_token == END_TOKEN:
      break
  return hin_sentence[0]

print(f"Evaluation translation {eng_sentence} : {hin_sentence}")
print("---------------------------------------------------------------------")


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [95]:
translation = translate("what should we do when the day starts?")
print(translation)

क्या क्या क्या का कि क्या करने का है?<END>


## Increasing num of epochs

In [96]:
from tqdm import tqdm
transformer.train()
#transformer.to(device)

total_loss = 0
num_epochs = 6

for epoch in range(num_epochs):
    print('Epoch', epoch)
    iterator = iter(train_loader)
    for batch_num, batch in tqdm(enumerate(iterator), total=len(iterator)):
        transformer.train()
        eng_batch, hin_batch = batch
        encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_mask(eng_batch, hin_batch)
        optim.zero_grad() # --> optim.zero_grad() is a method provided by PyTorch's optimizer classes (Adam used here)
        # that sets the gradients of all the model's parameters to zero.
        # It's an essential step to perform before computing gradients during backpropagation.
        hin_predictions = transformer(eng_batch,
                                      hin_batch,
                                      encoder_self_attention_mask,
                                      decoder_self_attention_mask,
                                      decoder_cross_attention_mask,
                                      encoder_start_token = False,
                                      encoder_end_token = False,
                                      decoder_start_token = True,
                                      decoder_end_token = True)
        labels = transformer.decoder.sentence_embedding.batch_tokenize(hin_batch, start_token = False, end_token = True)
        # criterian = nn.CrossEntropyLoss( ignore_index = hin_index[PADDING_TOKEN])
        loss = criterian(hin_predictions.view(-1, d_output),labels.view(-1)) # -1 is a place holder
        valid_indices = torch.where(labels.view(-1) == hin_index[PADDING_TOKEN], False, True) # <-- Binary Tensor
        loss = loss.sum()/valid_indices.sum() # <-- calculating the mean loss over the valid examples.
        loss.backward()
        # This line performs backpropagation to compute the gradients of the loss with respect to the model's parameters.
        #These gradients are essential for updating the model's parameters during the optimization step.
        optim.step() #<-- updates the model's parameters using an optimizer.
        # The optimizer uses the computed gradients to adjust the model's parameters in a way that minimizes the loss function
        if batch_num % 100 == 0:
            print(f'iteration {batch_num} : {loss.item()}')
            print(f"English: {eng_batch[0]}")
            print(f"Hindi Translation: {hin_batch[0]}")
            hin_sentence_predicted = torch.argmax(hin_predictions[0], axis= 1)
            predicted_sentence = ""
            for idx in hin_sentence_predicted:
                if idx == hin_index[END_TOKEN]:
                    break
                predicted_sentence += index_hin[idx.item()]
            print(f"Hindi Prediction: {predicted_sentence}")

save_path = '/content/drive/MyDrive/transformer_weights_6.pth'
torch.save({
    'model_state_dict': transformer.state_dict(),
    'optimizer_state_dict': optim.state_dict()
}, save_path)
print('weights saved in drive')

Epoch 0


  0%|          | 1/5194 [00:01<2:12:57,  1.54s/it]

iteration 0 : 0.0017205417389050126
English: mithali to anchor indian team against australia in odis
Hindi Translation: आस्ट्रेलिया के खिलाफ वनडे टीम की कमान मिताली को
Hindi Prediction: सरीट ट ड एा के लिलाफ किािशकीम के केा  किला   कर    ललर ल           ल ल     न         स  ल          र                          र ल   र   स    र रस       र   र             स   ल  ललस र  र सररस  ल ल लललल  सररलसलसससर सलससससलल  लस  स ललल  लर    सल स लल ल   ल  र  र    र   ल    र                         लललल  र ए  ललल     ल लललललसललल      ललललल ल  ललललललललललसल लललल


  2%|▏         | 101/5194 [02:39<2:20:11,  1.65s/it]

iteration 100 : 0.0018746809801086783
English: what does the modi government have?
Hindi Translation: मोदी सरकार के पास क्या है विकल्प
Hindi Prediction: कु े ककका  क् करसेक्या कै?काक् यत   ल  ल       र सलल लरल    ल   ल       लर  ब     ल स  प ल  प     स                       लल       प    ररर                            स   सररर सरर रल रर  सल ल लल   लललल  र ससम  रररलललरल स       ल ललल र              लरर  र    ल   लल    र रललर                   लर    म लए           ल    ललललरललसल ल   लललल ललल   लललल लल लस ल ललल


...

iteration 5000 : 0.0012054233811795712
English: new york, october 11
Hindi Translation: कालका, 11 अक्तूबर (निस)
Hindi Prediction: नोर,ात 1  फक्तूबर (निस)


 98%|█████████▊| 5101/5194 [2:19:47<02:40,  1.73s/it]

iteration 5100 : 0.0012551108375191689
English: the movie is directed by ss rajamouli.
Hindi Translation: यह फिल्म एसएस राजामौली द्वारा निर्देशित है।
Hindi Prediction: इह फिल्म ककीमएकेज्  र  हेवारा केर्देश त है।


100%|██████████| 5194/5194 [2:22:22<00:00,  1.64s/it]


weights saved in drive


In [97]:
checkpoint = torch.load('/content/drive/MyDrive/transformer_weights_6.pth')
transformer.load_state_dict(checkpoint['model_state_dict'])
optim.load_state_dict(checkpoint['optimizer_state_dict'])
print("Model weights loaded.")


transformer.eval()
def translate(eng_sentence):
  eng_sentence = (eng_sentence,)
  hin_sentence = ("",)
  for word_counter in range(max_sequence_length):
    encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_mask(eng_sentence, hin_sentence)
    predictions = transformer(eng_sentence,
                              hin_sentence,
                              encoder_self_attention_mask,
                              decoder_self_attention_mask,
                              decoder_cross_attention_mask,
                              encoder_start_token=False,
                              encoder_end_token=False,
                              decoder_start_token=True,
                              decoder_end_token=False)
    next_token_prob_distribution = predictions[0][word_counter]
    next_token_index = torch.argmax(next_token_prob_distribution).item()
    next_token = index_hin[next_token_index]
    hin_sentence = (hin_sentence[0] + next_token, )
    if next_token == END_TOKEN:
      break
  return hin_sentence[0]


Model weights loaded.


In [101]:
translation = translate("what should we do when the day starts?")
print(translation)

क्या है कि क्या करने के लिए क्या है?<END>


In [100]:
translation = translate("i hope this works.")
print(translation)

मुझे लगता है कि यह काम कर रहा है।<END>
